In [16]:
import sympy

In [17]:
#Defino todos los coeficientes del sistema
#que conforman a la matriz A
from sympy.abc import s # Variable laplace 
a,b,c,d= sympy.symbols("a,b,c,d")

A = sympy.Matrix([[a,b],[c,d]])
B = sympy.Matrix([[0],[1]])
# No se bien porque sympy arma a C como una matrix de 3x1
# Si la transpongo la convierte en una fila y queda bien representado
# El sistema
C = sympy.Matrix([1,0]).T


El calculo de la funcion de transferencia se hace a través de la formula:
$$
 H(s) = C{(sI - A)^{-1}}B + D
$$


In [18]:
#A es siempre cuadrada, por eso puedo agarrar las rows o las columns
#indistintamente para saber la dimension de la diagonal
temp = s * sympy.eye(A.shape[0]) - A
inv = temp.inv()

In [39]:
temp = C * inv 
H = temp * B
transfer_function = H[0]
n,d = sympy.fraction(sympy.factor(transfer_function))

##This is useful for jordan form
tf_expr = n/d

from sympy.physics.control.lti import TransferFunction
# Si bien, el calculo ya fue realizado, y transfer_function tiene
# la forma de la funcion de transferencia, para la libreria 
# no es mas que una expresion mas. 
# Al usar la clase TransferFuncion podemos despues acceder a metodos
# utiles para operar (como puede ser el calculo de polos)
tf = TransferFunction(n,d,s)
tf

TransferFunction(b, a*d - a*s - b*c - d*s + s**2, s)

In [20]:
import functools 

den_args_str = [str(x) for x in d.args]
print(den_args_str)
pow2 = [x for x in den_args_str if "s**2" in x]
pow1 = [x for x in den_args_str if "*s" in x and x not in pow2] #Son los terminos que tienen s ^ 1
tail_terms = [x for x in den_args_str if x not in pow1 and x not in pow2] # Son los terminos sin s
 

# Armo las expresiones que despues se introducen en la matriz A 
# concatenando los valores de pow1 y tail_terms
pow1_expr =sympy.factor(sympy.parse_expr(functools.reduce(lambda a,b: a + b, pow1), evaluate=False))
tail_terms_expr = sympy.factor(sympy.parse_expr(functools.reduce(lambda a,b: a + b, tail_terms), evaluate=False))

# Elimino las variable de la expresion, ya que solo me interesa
# quedrme con los factores que multiplican a la variable
def sanitize_expr_str(e):
    start = e.find("s")
    if start == -1:
        return e 
    return e.replace(e[start:start+2:1], "")

# Estos van a ir a la matiz a en la ultima fila
a_0 = sympy.parse_expr(sanitize_expr_str(str(tail_terms_expr)), evaluate=False)
a_1 =sympy.parse_expr(sanitize_expr_str(str(pow1_expr)), evaluate=False)



['s**2', 'a*d', '-a*s', '-b*c', '-d*s']


### Forma de control
Es deducida directamente desde los terminos de la funcion


In [21]:


A_ctrl = sympy.Matrix([[0,1], [a_0, a_1]])
B_ctrl = sympy.Matrix([[0], [1]])
C_ctrl = sympy.Matrix([n,0]).T

In [22]:
A_ctrl

Matrix([
[        0,      1],
[a*d - b*c, -a - d]])

In [23]:
B_ctrl

Matrix([
[0],
[1]])

In [24]:
C_ctrl

Matrix([[b, 0]])

## Forma observable
Transponemos el A de control e intercambiamos B y C de control

In [25]:

import copy

A_obs = copy.deepcopy(A_ctrl).T
B_obs = copy.deepcopy(C_ctrl)
C_obs = copy.deepcopy(B_ctrl).T


In [26]:
A_obs

Matrix([
[0, a*d - b*c],
[1,    -a - d]])

In [27]:
B_obs

Matrix([[b, 0]])

In [28]:
C_obs

Matrix([[0, 1]])

In [30]:
tf_poles = tf.poles()


[a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2,
 a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2]

In [31]:
tf_poles[0]


a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2

In [32]:
tf_poles[1]


a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2

## Forma de Jordan

In [34]:
## Para esta forma de jordan, vamos a asumir que hay todos polos reales y distintos 
## A la hora de definir valores para todos los parametros, podemos concluir
## si hace falta modificarlo 
A_jordan = sympy.Matrix([[tf_poles[0], 0], [0,tf_poles[1]]])
A_jordan

Matrix([
[a/2 + d/2 - sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2,                                               0],
[                                              0, a/2 + d/2 + sqrt(a**2 - 2*a*d + 4*b*c + d**2)/2]])

In [54]:
B_jordan = sympy.ones(len(tf_poles),1)
B_jordan

Matrix([
[1],
[1]])

In [50]:
#residue calcula el residuo de las fracciones parciales para la funcion
#de transferencia
C_jordan = [sympy.residue(tf_expr, s, tf_poles[0]), sympy.residue(tf_expr, s, tf_poles[1])]
C_jordan

[-b/sqrt(a**2 - 2*a*d + 4*b*c + d**2), b/sqrt(a**2 - 2*a*d + 4*b*c + d**2)]